# Used Code for the Thesis:

# Dark Energy: Constraining Cosmological Models with Extra Spatial Dimensions Based on Type Ia Supernovae Data

## $\alpha$DGP Model

### Fitting the Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve, curve_fit
from scipy.integrate import quad

# Lichtgeschwindigkeit in km/s
c = 299792.458

# Funktion f(u) im DGP-Modell
def f_u(u, Omega_m, z, alpha):
    return u**2 - (1 - Omega_m) * u**alpha - Omega_m * (1 + z)**3

# Löse f(u) = 0 für gegebenes z
def solve_u(Omega_m, z, alpha):
    u_init = np.sqrt(Omega_m * (1 + z)**3 + (1 - Omega_m))  # sinnvoller Startwert
    u_sol = fsolve(f_u, u_init, args=(Omega_m, z, alpha))[0]
    return u_sol

# Hubble-Parameter im DGP-Modell: H(z) = u(z) * H_0
def H_DGP(z, Omega_m, H_0, alpha):
    u = solve_u(Omega_m, z, alpha)
    return u * H_0

# Leuchtkraftentfernung im DGP-Modell
def d_L(z, Omega_m, H_0, alpha):
    integrand = lambda ztilde: 1.0 / H_DGP(ztilde, Omega_m, H_0, alpha)
    integral, _ = quad(integrand, 0, z)
    return c * (1 + z) * integral

# Modellfunktion für relative Magnitude
def model_func(z, Omega_m, H_0, alpha):
    d_L_vals = np.array([d_L(zi, Omega_m, H_0, alpha) for zi in np.atleast_1d(z)])
    return 5 * np.log10(d_L_vals) + 25

# Funktion zum Laden der Pantheon+SH0ES-Daten
def load_pantheon_sh0es_data(data_file, cov_file):
    dat = pd.read_csv(data_file, sep=' ')
    z = np.array(dat['zHD'])
    dm = np.array(dat['MU_SH0ES'])

    ncol = 1701
    cov1 = np.loadtxt(cov_file)
    cov1.shape = (ncol, ncol)

    selnotlocal = z > 0.01
    select = selnotlocal

    z = z[select]
    dm = dm[select]
    thecov = cov1[select, :]
    thecov = thecov[:, select]

    return z, dm, thecov

# ======= HIER BEGINNT DAS HAUPTPROGRAMM =======

# Pfade anpassen
data_file = "/Users/insertdatapath/Pantheon+SH0ES.dat"
cov_file = "/Users/insertdatapath/Pantheon+SH0ES_STAT+SYS.cov"

# Daten laden
z_pantheon, dm_pantheon, cov_matrix_pantheon = load_pantheon_sh0es_data(data_file, cov_file)

# Fehler berechnen (Diagonalelemente der Kovarianzmatrix)
dm_errors = np.sqrt(np.diag(cov_matrix_pantheon))

# Anfangsschätzungen: [Omega_m, H_0, alpha]
initial_guess = [0.3, 70.0, 0.0]

# Curve Fit mit Unsicherheiten
popt, pcov = curve_fit(
    model_func,
    z_pantheon,
    dm_pantheon,
    sigma=dm_errors,
    p0=initial_guess,
    absolute_sigma=True,
    maxfev=10000 #mehr Schritte zur Findung einer Lsg. erlauben
)

omega_m_fit, h_0_fit, alpha_fit = popt

# Ergebnisse ausgeben
print(f"Beste Werte: Omega_M = {omega_m_fit:.3f}, H_0 = {h_0_fit:.3f}, alpha = {alpha_fit:.3f}")

# Plot erstellen
plt.figure(figsize=(10, 6))
plt.errorbar(z_pantheon,
             dm_pantheon,
             yerr=dm_errors,
             fmt='o',
             color='red',
             label="Pantheon+SHOES Data")

# Fitkurve plotten
z_fit_values = np.linspace(min(z_pantheon), max(z_pantheon), 500)
dm_fit_values = model_func(z_fit_values, omega_m_fit, h_0_fit, alpha_fit)
plt.plot(z_fit_values,
         dm_fit_values,
         label=f"Fit ($\\Omega_m$={omega_m_fit:.3f}, $H_0$={h_0_fit:.3f}, $\\alpha$={alpha_fit:.3f})",
         color='blue')

plt.xlabel("Redshift $z$")
plt.ylabel("Relative Magnitude $m_{rel}$")
plt.title(r"Pantheon+SHOES: Relative Magnitude vs. Redshift Fit ($\alpha$DGP Model)")
plt.savefig('m_rel_vs_z_fit_DGP', dpi=300)
plt.legend()
plt.grid()
plt.show()

### Contour Plot for Uncertainies

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy.integrate import quad, simps
import warnings
from tqdm import tqdm
import os

# --- Konstanten ---
c = 299792.458  # Lichtgeschwindigkeit in km/s

# --- Fehlerbehandlung für numerische Berechnungen ---
def safe_fsolve(func, u_init, args=()):
    try:
        result = fsolve(func, u_init, args=args)
        if np.any(np.isnan(result)) or np.any(np.isinf(result)):
            raise ValueError("fsolve returned NaN or Inf")
        return result[0]
    except Exception as e:
        print(f"Fehler bei fsolve: {e}")
        return np.nan

def safe_quad(integrand, a, b, args=()):
    try:
        result, _ = quad(integrand, a, b, args=args, limit=100)
        if np.isnan(result) or np.isinf(result):
            raise ValueError("quad returned NaN or Inf")
        return result
    except Exception as e:
        print(f"Fehler bei quad: {e}")
        return np.nan

# --- DGP Modellfunktionen ---
def f_u(u, Omega_m, z, alpha):
    return u**2 - (1 - Omega_m) * u**alpha - Omega_m * (1 + z)**3

def solve_u(Omega_m, z, alpha):
    u_init = np.sqrt(Omega_m * (1 + z)**3 + (1 - Omega_m))
    u_sol = safe_fsolve(f_u, u_init, args=(Omega_m, z, alpha))
    return u_sol

def H_DGP(z, Omega_m, H_0, alpha):
    u = solve_u(Omega_m, z, alpha)
    if np.isnan(u):
        return np.nan
    return u * H_0

# --- d_L (ohne H_0) ---
def d_L(z, Omega_m, alpha):
    integrand = lambda ztilde: 1.0 / solve_u(Omega_m, ztilde, alpha)
    integral = safe_quad(integrand, 0, z)
    if np.isnan(integral):
        return np.nan
    return c * (1 + z) * integral  # in Mpc

# --- Marginalisiertes chi² über 𝓜 ---
def marginalized_chi2(z, m_obs, m_err, Omega_m, alpha):
    d_Ls = np.array([d_L(zi, Omega_m, alpha) for zi in z])
    if np.any(np.isnan(d_Ls)) or np.any(d_Ls <= 0):
        return np.inf
    
    log_d_Ls = 5 * np.log10(d_Ls)

    # Gitter in 𝓜 (geschwungenes M)
    M_vals = np.linspace(15, 16, 200) #sinnvoll für wahrscheinliche H_0: 63.095 (16) -100(15)
    chi2_vals = []

    for M in M_vals:
        model_mags = M + log_d_Ls
        chi2 = np.sum(((m_obs - model_mags) / m_err) ** 2)
        chi2_vals.append(np.exp(-0.5 * chi2))

    integral = simps(chi2_vals, M_vals)
    if integral <= 0 or not np.isfinite(integral):
        return np.inf

    chi2_marg = -2 * np.log(integral)
    return chi2_marg

# --- Pantheon+SH0ES Daten laden ---
def load_pantheon_sh0es_data(data_file, cov_file):
    dat = pd.read_csv(data_file, delim_whitespace=True)
    z = np.array(dat['zHD'])
    dm = np.array(dat['MU_SH0ES'])

    ncol = len(dat)
    cov1 = np.loadtxt(cov_file)
    cov1.shape = (ncol, ncol)

    select = z > 0.01
    z = z[select]
    dm = dm[select]
    thecov = cov1[select, :]
    thecov = thecov[:, select]
    return z, dm, thecov

# --- Fehlerbehandlung für große Gitter ---
def run_large_grid(omega_m_vals, alpha_vals, z_pantheon, mu_pantheon, mu_errors, checkpoint_interval=10):
    chi2_grid = np.zeros((len(omega_m_vals), len(alpha_vals)))

    # Raster durchlaufen
    for i, Omega_m in tqdm(enumerate(omega_m_vals), total=len(omega_m_vals), desc="Berechne χ²-Gitter"):
        for j, alpha in enumerate(alpha_vals):
            try:
                chi2 = marginalized_chi2(z_pantheon, mu_pantheon, mu_errors, Omega_m, alpha)
                chi2_grid[i, j] = chi2
            except Exception as e:
                print(f"Fehler bei Omega_m={Omega_m}, alpha={alpha}: {e}")
                chi2_grid[i, j] = np.inf

            # Speichern der Zwischenergebnisse
            if (i * len(alpha_vals) + j) % checkpoint_interval == 0:
                np.save("chi2_grid_checkpoint.npy", chi2_grid)
                print(f"Zwischenergebnis gespeichert: {i}, {j}")

    return chi2_grid

# --- Daten einlesen ---
data_file = "/Users/insertdatapath/Pantheon+SH0ES.dat"
cov_file = "/Users/insertdatapath/Pantheon+SH0ES_STAT+SYS.cov"
z_pantheon, mu_pantheon, cov_matrix_pantheon = load_pantheon_sh0es_data(data_file, cov_file)
mu_errors = np.sqrt(np.diag(cov_matrix_pantheon))

# --- Parameter-Raster ---
omega_m_vals = np.linspace(0.1, 0.6, 100)
alpha_vals = np.linspace(-5.0, 1.9, 100)

# --- Berechnungen ---
chi2_grid = run_large_grid(omega_m_vals, alpha_vals, z_pantheon, mu_pantheon, mu_errors)

# --- Ergebnis abspeichern ---
np.save("chi2_grid.npy", chi2_grid)
np.save("omega_m_vals.npy", omega_m_vals)
np.save("alpha_vals.npy", alpha_vals)
print("Gitter gespeichert.")

# Wenn der Prozess unterbrochen wurde, Zwischenergebnis wieder laden über:
# chi2_grid = np.load("chi2_grid_checkpoint.npy")

# --- Plotting vorbereiten ---
Alpha_mesh, Omega_m_mesh = np.meshgrid(alpha_vals, omega_m_vals)

# --- Mindestwert für Chi² extrahieren ---
min_chi2 = np.nanmin(chi2_grid)

# --- Contour levels (1σ, 2σ, 3σ), ggf. durch dof teilen, wenn du das willst ---
# Falls du Δχ² pro Freiheitsgrad willst, musst du `dof` definieren.
# Ansonsten hier wie im Original ohne dof teilen:
contour_levels = [min_chi2 + delta for delta in [2.30, 6.17, 11.8]]

# --- Dynamische Achsenskalierung ---
relevant_indices = np.where((chi2_grid >= contour_levels[0]) & (chi2_grid <= contour_levels[-1]))

# Vorsicht: Hier muss man schauen, dass keine leeren Arrays entstehen
if relevant_indices[0].size > 0 and relevant_indices[1].size > 0:
    min_alpha = alpha_vals[relevant_indices[1]].min()
    max_alpha = alpha_vals[relevant_indices[1]].max()
    min_omega_m = omega_m_vals[relevant_indices[0]].min()
    max_omega_m = omega_m_vals[relevant_indices[0]].max()
else:
    # Fallback wenn keine relevanten Indizes gefunden wurden
    min_alpha, max_alpha = alpha_vals.min(), alpha_vals.max()
    min_omega_m, max_omega_m = omega_m_vals.min(), omega_m_vals.max()

# --- Plot erstellen ---
plt.figure(figsize=(10, 8))

# Hintergrund des gesamten Plots auf weiß setzen
plt.rcParams['axes.facecolor'] = 'white'  # Hintergrund der Achsen
plt.gcf().set_facecolor('white')  # Hintergrund des gesamten Plots

# Meshgrid für Konturplot
Alpha_mesh, Omega_m_mesh = np.meshgrid(alpha_vals, omega_m_vals)

# Nur Konturlinien (keine Farbfüllung)
contour_colors = ['blue', 'green', 'red']
lines = []
labels = [r'1$\sigma$ (Δ$\chi^2$=2.3)', r'2$\sigma$ (Δ$\chi^2$=6.17)', r'3$\sigma$ (Δ$\chi^2$=11.8)']

# Konturen für die Standardabweichungen
for level, color in zip(contour_levels, contour_colors):
    # Berechne die Konturen (nur Linien, keine Füllung)
    contour = plt.contour(Alpha_mesh,
                           Omega_m_mesh,
                           chi2_grid,
                           levels=[level],
                           colors=[color],
                           linewidths=2)

    # Füge die Linien aus den Konturen zur Legende hinzu
    lines.append(lines)  # Füge die erste Linie der Konturen zur Legende hinzu

# Legende erstellen
plt.legend(lines, labels, loc='upper right', fontsize=12, frameon=True)

# Achsentitel und Formatierung
plt.xlabel(r'$\alpha$', fontsize=14)
plt.ylabel(r'$\Omega_m$', fontsize=14)
plt.title(r'$\chi^2$ Contours for $\Omega_m$ and $\alpha$', fontsize=18)

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)

# Dynamische Achsenskalierung
plt.xlim(min_alpha - 0.1, max_alpha + 0.1)
plt.ylim(min_omega_m - 0.01, max_omega_m + 0.01)

# Speichern und Anzeigen
plt.tight_layout()
plt.savefig('chi2_contours_DGP_numerical_pretty.png', dpi=300)
plt.show()

# --- Best-Fit-Werte extrahieren ---
min_index = np.unravel_index(np.argmin(chi2_grid), chi2_grid.shape)
best_omega_m = omega_m_vals[min_index[0]]
best_alpha = alpha_vals[min_index[1]]

# --- Ergebnisse ausgeben ---
print("Best-fit parameter values:")
print(f"Omega_m = {best_omega_m:.4f}")
print(f"alpha   = {best_alpha:.4f}")

### MCMC Contour Plots

In [ ]:
import numpy as np
import emcee
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy.integrate import quad
from scipy.ndimage import gaussian_filter
from tqdm import tqdm
from matplotlib.lines import Line2D

# --- Konstanten ---
c = 299792.458  # Lichtgeschwindigkeit [km/s]

# --- DGP Modellfunktionen ---
def f_u(u, Omega_m, z, alpha):
    return u**2 - (1 - Omega_m) * u**alpha - Omega_m * (1 + z)**3

def solve_u(Omega_m, z, alpha):
    u_init = np.sqrt(Omega_m * (1 + z)**3 + (1 - Omega_m))
    return fsolve(f_u, u_init, args=(Omega_m, z, alpha))[0]

def H_DGP(z, Omega_m, H_0, alpha):
    u = solve_u(Omega_m, z, alpha)
    return u * H_0

def d_L(z, Omega_m, H_0, alpha):
    integrand = lambda ztilde: 1.0 / H_DGP(ztilde, Omega_m, H_0, alpha)
    integral, _ = quad(integrand, 0, z, limit=100)
    return c * (1 + z) * integral

def model_func(z, Omega_m, H_0, alpha):
    return 5 * np.log10([d_L(zi, Omega_m, H_0, alpha) for zi in z]) + 25

# --- Log-Likelihood ---
def log_likelihood(params, z, dm, cov_matrix):
    Omega_m, H_0, alpha = params
    try:
        mag_model = model_func(z, Omega_m, H_0, alpha)
        delta_m = dm - mag_model
        inv_cov = np.linalg.inv(cov_matrix)
        chi2 = delta_m.T @ inv_cov @ delta_m
        return -0.5 * chi2
    except:
        return -np.inf

# --- Log-Prior ---
def log_prior(params):
    Omega_m, H_0, alpha = params
    if 0.05 < Omega_m < 0.75 and 50 < H_0 < 90 and -5 < alpha < 2:
        return 0.0
    return -np.inf

# --- Log-Posterior ---
def log_posterior(params, z, dm, cov_matrix):
    lp = log_prior(params)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(params, z, dm, cov_matrix)

# --- Daten einlesen ---
def load_pantheon_sh0es_data(data_file, cov_file):
    dat = pd.read_csv(data_file, delim_whitespace=True)
    z = np.array(dat['zHD'])
    dm = np.array(dat['MU_SH0ES'])

    ncol = len(dat)
    cov1 = np.loadtxt(cov_file)
    cov1.shape = (ncol, ncol)

    select = z > 0.01
    z = z[select]
    dm = dm[select]
    thecov = cov1[select][:, select]
    return z, dm, thecov

# --- Dateipfade ---
data_file = "/Users/insertdatapath/Pantheon+SH0ES.dat"
cov_file = "/Users/insertdatapath/Pantheon+SH0ES_STAT+SYS.cov"
# --- Daten laden ---
z_pantheon, mu_pantheon, cov_matrix_pantheon = load_pantheon_sh0es_data(data_file, cov_file)

# --- emcee ---
ndim = 3  # Anzahl der Parameter: Omega_m, H_0, alpha
nwalkers = 30
nsteps = 700
initial_guess = [0.3, 73.0, 0.0]
pos = initial_guess + 1e-1 * np.random.randn(nwalkers, ndim)
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_posterior, args=(z_pantheon, mu_pantheon, cov_matrix_pantheon))
sampler.run_mcmc(pos, nsteps, progress=True)
samples = sampler.get_chain(discard=160, flat=True)
Omega_m_samples, H_0_samples, alpha_samples = samples[:, 0], samples[:, 1], samples[:, 2]

# Visualisierung der Startpositionen der Walker für Omega_m und alpha
plt.figure(figsize=(10, 6))
plt.scatter(pos[:, 0], pos[:, 2], label='Walker Startpositionen')  # Omega_m vs. alpha
plt.xlabel(r'$\Omega_m$', fontsize=14)
plt.ylabel(r'$\alpha$', fontsize=14)
plt.title('Startpositionen der Walker (Omega_m vs. alpha)', fontsize=16)
plt.grid(True)
plt.show()

# --- Beste Parameter (Maximum Likelihood) finden ---
log_likelihoods = np.array([log_likelihood(params, z_pantheon, mu_pantheon, cov_matrix_pantheon) for params in samples])
best_index = np.argmax(log_likelihoods)
best_params = samples[best_index]
best_Omega_m, best_H_0, best_alpha = best_params
chi2_min = -2 * log_likelihoods[best_index]

# Posterior Plot: Omega_m vs Alpha
nbins = 70
omegam_range = (0.01, 0.8)
alpha_range = (-5, 1.99)

H, xedges, yedges = np.histogram2d(alpha_samples, Omega_m_samples, bins=nbins,  # Achsen getauscht
                                    range=[alpha_range, omegam_range], density=True)  # Achsen getauscht
H_smooth = gaussian_filter(H, sigma=1.0)
xbin = 0.5 * (xedges[:-1] + xedges[1:])
ybin = 0.5 * (yedges[:-1] + yedges[1:])
X, Y = np.meshgrid(xbin, ybin)

# Konturen für Glaubwürdigkeitsniveaus
sorted_H = np.sort(H_smooth.flatten())[::-1]
cumulative = np.cumsum(sorted_H)
cumulative /= cumulative[-1]
cred_levels = [0.683, 0.954, 0.997]
contour_levels = [sorted_H[np.searchsorted(cumulative, level)] for level in cred_levels]
contour_levels = np.sort(contour_levels)

# Plot erstellen
plt.figure(figsize=(10, 8))
plt.contourf(X, Y, H_smooth.T, levels=50, cmap='Blues', alpha=0.7)
contour_colors = ['red', 'green', 'blue']
labels = [r'99.7% credible region (3σ)', r'95.4% credible region (2σ)', r'68.3% credible region (1σ)']
lines = []

for level, color in zip(contour_levels, contour_colors):
    line = plt.contour(X, Y, H_smooth.T, levels=[level], colors=[color], linewidths=2)
    lines.append(line.legend_elements()[0][0])

# Best-fit Parameter anzeigen
best_label = fr'Best Fit: $\Omega_m = {best_Omega_m:.3f}$, $\alpha = {best_alpha:.3f}$'
dummy_line = Line2D([0], [0], color='white', alpha=0)
lines.append(dummy_line)
labels.append(best_label)

plt.legend(lines, labels, loc='upper right', fontsize=12)
plt.title(r'Contour Plots of $\Omega_m$ and $\alpha$ via Monte Carlo', fontsize=18)
plt.xlabel(r'$\alpha$', fontsize=14)  # Achsenbeschriftung für Alpha
plt.ylabel(r'$\Omega_m$', fontsize=14)  # Achsenbeschriftung für Omega_m
plt.xlim(alpha_range)
plt.ylim(omegam_range)
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.savefig("contour_plot_MCMC_DGP_emcee_.png", dpi=300)  
plt.show()


# Ergebnisse speichern
np.savez("dgp_mcmc_results_emcee.npz",
         Omega_m_samples=Omega_m_samples,
         H_0_samples=H_0_samples,
         alpha_samples=alpha_samples,
         best_params=best_params,
         chi2_min=chi2_min)
print("Ergebnisse gespeichert in 'dgp_mcmc_results_emcee.npz'")

### Histograms

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize, fsolve
from scipy.integrate import quad
from tqdm import tqdm

# --------------------------------------------
# Daten einlesen & vorbereiten
# --------------------------------------------

# Lade Pantheon+SH0ES-Daten
dat = pd.read_csv('/Users/insertdatapath/Pantheon+SH0ES.dat', sep=' ')
z = np.array(dat['zHD'])
dm = np.array(dat['MU_SH0ES'])

# Lade Kovarianzmatrix
ncol = 1701
cov1 = np.loadtxt("/Users/insertdatapath/Pantheon+SH0ES_STAT+SYS.cov")
cov1.shape = (ncol, ncol)
thecov = cov1

# Filtere lokale Supernovae (nur z > 0.01)
select = z > 0.01
z_filtered = z[select]
dm_filtered = dm[select]
thecov_filtered = thecov[select, :][:, select]

# Für später
z = z_filtered
dm = dm_filtered
cov_matrix = thecov_filtered

# --------------------------------------------
# Kosmologie: DGP-Modellfunktionen
# --------------------------------------------

c = 299792.458  # Lichtgeschwindigkeit in km/s

def f_u(u, Omega_m, z, alpha):
    return u**2 - (1 - Omega_m) * u**alpha - Omega_m * (1 + z)**3

def solve_u(Omega_m, z, alpha):
    u_init = np.sqrt(Omega_m * (1 + z)**3 + (1 - Omega_m))
    return fsolve(f_u, u_init, args=(Omega_m, z, alpha))[0]

def H_DGP(z, Omega_m, H_0, alpha):
    u = solve_u(Omega_m, z, alpha)
    return u * H_0

def d_L(z, Omega_m, H_0, alpha):
    integrand = lambda ztilde: 1.0 / H_DGP(ztilde, Omega_m, H_0, alpha)
    integral, _ = quad(integrand, 0, z, limit=100)
    return c * (1 + z) * integral

def model_func(z_array, Omega_m, H_0, alpha):
    return 5 * np.log10([d_L(zi, Omega_m, H_0, alpha) for zi in z_array]) + 25

# --------------------------------------------
# Chi²-Funktion
# --------------------------------------------

inv_cov_matrix = np.linalg.inv(cov_matrix)

def chi_squared(params, z, dm, inv_cov_matrix):
    Omega_m, H_0, alpha = params
    mag_model = model_func(z, Omega_m, H_0, alpha)
    delta_m = dm - mag_model
    return delta_m.T @ inv_cov_matrix @ delta_m


# --------------------------------------------
# Monte Carlo Simulation
# --------------------------------------------

def simulate_supernovae(z, Omega_m, H_0, alpha, cov_matrix):
    dm_model = model_func(z, Omega_m, H_0, alpha)
    error = np.random.multivariate_normal(np.zeros(len(z)), cov_matrix)
    dm_sim = dm_model + error
    return dm_sim, cov_matrix

# Startparameter
Omega_m_true = 0.247
H0_true = 72.890
alpha_true = 0.905
n_simulations = 150

# Ergebnisse speichern
fitted_omegam = []
fitted_h0 = []
fitted_alpha = []

# Monte Carlo Simulation starten
print(f"Starte Monte Carlo Simulation für DGP-Modell (Ωm={Omega_m_true:.3f}, H0={H0_true:.2f}, alpha={alpha_true})")

for _ in tqdm(range(n_simulations)):
    dm_sim, cov_sim = simulate_supernovae(z, Omega_m_true, H0_true, alpha_true, cov_matrix)

    result = minimize(
        chi_squared,
        x0=[0.3, 70, 0.0],  # Startwerte für [Omega_m, H0, alpha]
        args=(z, dm_sim, cov_sim),
        bounds=[(0.1, 0.6), (50, 90), (-5, 1.999)],  
        method='L-BFGS-B'
    )

    if result.success:
        omega_fit, h0_fit, alpha_fit = result.x
        fitted_omegam.append(omega_fit)
        fitted_h0.append(h0_fit)
        fitted_alpha.append(alpha_fit)

# --------------------------------------------
# Ergebnisse visualisieren
# --------------------------------------------

fitted_omegam = np.array(fitted_omegam)
fitted_h0 = np.array(fitted_h0)
fitted_alpha = np.array(fitted_alpha)

plt.figure(figsize=(18, 5))

# Ωm Histogramm
plt.subplot(1, 3, 1)
plt.hist(fitted_omegam, bins=30, color='skyblue', edgecolor='black')
plt.axvline(np.mean(fitted_omegam), color='red', linestyle='--', label=f"Mean = {np.mean(fitted_omegam):.3f}")
plt.xlabel(r'Fitted $\Omega_m$')
plt.ylabel('Incidence')
plt.title(r'Distribution of $\Omega_m$')
plt.legend()

# H0 Histogramm
plt.subplot(1, 3, 2)
plt.hist(fitted_h0, bins=30, color='lightgreen', edgecolor='black')
plt.axvline(np.mean(fitted_h0), color='red', linestyle='--', label=f"Mean = {np.mean(fitted_h0):.2f}")
plt.xlabel(r'Fitted $H_0$')
plt.ylabel('Incidence')
plt.title('Distribution of $H_0$')
plt.legend()

# alpha Histogramm
plt.subplot(1, 3, 3)
plt.hist(fitted_alpha, bins=30, color='orange', edgecolor='black')
plt.axvline(np.mean(fitted_alpha), color='red', linestyle='--', label=f"Mean = {np.mean(fitted_alpha):.2f}")
plt.xlabel(r'Fitted $\alpha$')
plt.ylabel('Incidence')
plt.title(r'Distribution of $\alpha$')
plt.legend()

plt.tight_layout()
plt.show()

# --------------------------------------------
# Ergebnisse speichern
# --------------------------------------------

np.savez("montecarlo_histo_DGP_results.npz",
         fitted_omegam=fitted_omegam,
         fitted_h0=fitted_h0,
         fitted_alpha=fitted_alpha,
         z=z,
         dm=dm,
         cov_matrix=cov_matrix,
         Omega_m_true=Omega_m_true,
         H0_true=H0_true,
         alpha_true=alpha_true,
         n_simulations=n_simulations)

print("Simulationsergebnisse gespeichert in 'montecarlo_histo_DGP_results.npz'")